In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy

import sys
sys.path.append("../../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-07-30 15:21:47 __main__ INFO     torch.__version__='2.3.1', torch.version.cuda='12.1'
2024-07-30 15:21:47 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2024-07-30 15:21:47 __main__ INFO     transformers.__version__='4.42.4'


In [3]:
from dataclasses_json import DataClassJsonMixin
from dataclasses import dataclass, field, fields
from typing import Optional
import random
from src.dataset import BridgeSample, BridgeRelation, BridgeDataset
from src.dataset import load_bridge_relation, load_bridge_relations, load_bridge_dataset        

In [4]:
relations = load_bridge_relations()
dataset = BridgeDataset(relations)

2024-07-30 15:21:47 src.dataset INFO     initialized bridge relation superpower_characters with 27 examples
2024-07-30 15:21:47 src.dataset INFO     initialized bridge relation sport_players with 23 examples
2024-07-30 15:21:47 src.dataset INFO     initialized bridge relation movie_actor with 52 examples
2024-07-30 15:21:47 src.dataset INFO     initialized bridge relation architect_building with 21 examples
2024-07-30 15:21:47 src.dataset INFO     initialized bridge dataset with 4 relations and 119 examples


In [5]:
print(dataset[5][0])

Given two entities, find a common link between them.
#
What is a common link between Captain America and Deathstroke?
A: super soldier - an attribute that both characters Captain America and Deathstroke possess.
#
What is a common link between Simone Biles and Nadia Comaneci?
A: gymnastics - a sport where both players Simone Biles and Nadia Comaneci are known for.
#
What is a common link between Gael García Bernal and Che Guevara?
A: The Motorcycle Diaries - a movie where Gael García Bernal played the role of Che Guevara.
#
What is a common link between Louvre Pyramid and Bank of China Tower?
A: I. M. Pei - who was the architect of both buildings Louvre Pyramid and Bank of China Tower.
#
What is a common link between Michael Vaughan and Sourav Ganguly?
A:


In [6]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer

# model_key = "meta-llama/Meta-Llama-3-8B"
model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "Qwen/Qwen2-7B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-07-30 15:21:47 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.29s/it]

2024-07-30 15:21:53 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-8B-Instruct> | size: 15316.516 MB | dtype: torch.float16 | device: cuda:0


In [7]:
sample_idx = 17

prompt, answer = dataset[sample_idx]
answer

'Walk the Line - a movie where Joaquin Phoenix played the role of Johnny Cash.'

In [8]:
from src.models import prepare_input

# inputs = prepare_input(
#     prompts = prompt,
#     tokenizer=mt,
#     add_bos_token=False
# )

# generation = mt._model.generate(
#     **inputs,
#     max_new_tokens=100,
#     top_k = 1
# )

# print(mt.tokenizer.decode(generation[0], skip_special_tokens=False))

In [9]:
from src.functional import predict_bridge_entity

predicted_ans = predict_bridge_entity(mt, prompt)
predicted_ans

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


' Walk the Line - a movie where Joaquin Phoenix played the role of Johnny Cash.\n'

In [10]:
from src.functional import ask_gpt4o

query_sample = dataset.examples[sample_idx]
ask_gpt4o(query_sample, predicted_ans)

2024-07-30 15:22:00 openai._base_client DEBUG    Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '\nA smaller language model was asked the following question:\n"What is a common link between Joaquin Phoenix and Johnny Cash?"\nAnd the model gave the following answer:\n"Walk the Line - a movie where Joaquin Phoenix played the role of Johnny Cash."\nIs it correct? Your answer should start with "Yes" or "No". If the answer is "Yes", don\'t say anything else. If the answer is "No", give explanation why.\n'}], 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0}}
2024-07-30 15:22:00 openai._base_client DEBUG    Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2024-07-30 15:22:00 httpcore.connection DEBUG    connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
2024-07-30 15

'Yes'

In [13]:
from src.functional import filter_bridge_samples_by_model_knowledge

dataset = filter_bridge_samples_by_model_knowledge(mt, dataset)

  0%|          | 0/119 [00:00<?, ?it/s]

2024-07-30 15:23:34 src.functional DEBUG    found cached gpt4o response for b012d2090ae14428fedf73e25cd713f9 - loading
2024-07-30 15:23:34 src.functional INFO     ['Iceman', 'Killer Frost'] <> ice control | predicted:  DC Comics - both characters Iceman and Killer Frost are part of the DC Comics universe.
 => (✗)


  1%|          | 1/119 [00:06<13:36,  6.92s/it]

2024-07-30 15:23:40 src.functional DEBUG    found cached gpt4o response for 19123a86bec093698cfa8554d112e4b2 - loading
2024-07-30 15:23:40 src.functional INFO     ['Human Torch', 'Firestorm'] <> fire control | predicted:  Fire manipulation - a superpower that both characters Human Torch and Firestorm possess.
 => (✓)


  2%|▏         | 2/119 [00:13<12:42,  6.52s/it]

...

2024-07-30 15:39:04 openai._base_client DEBUG    Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '\nA smaller language model was asked the following question:\n"What is a common link between 30 St Mary Axe (The Gherkin) and Hearst Tower?"\nAnd the model gave the following answer:\n"Norman Foster - who was the architect of both buildings 30 St Mary Axe (The Gherkin) and Hearst Tower."\nIs it correct? Your answer should start with "Yes" or "No". If the answer is "Yes", don\'t say anything else. If the answer is "No", give explanation why.\n'}], 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0}}
2024-07-30 15:39:04 openai._base_client DEBUG    Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2024-07-30 15:39:04 httpcore.connection DEBUG    close.started
2024-07-30 15:39:04 httpcore.connection DEBUG    close.complet

 99%|█████████▉| 118/119 [15:38<00:09,  9.71s/it]

2024-07-30 15:39:11 openai._base_client DEBUG    Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '\nA smaller language model was asked the following question:\n"What is a common link between Bradley Cooper and Chris Kyle?"\nAnd the model gave the following answer:\n"American Sniper - a movie where Bradley Cooper played the role of Chris Kyle."\nIs it correct? Your answer should start with "Yes" or "No". If the answer is "Yes", don\'t say anything else. If the answer is "No", give explanation why.\n'}], 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0}}
2024-07-30 15:39:11 openai._base_client DEBUG    Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2024-07-30 15:39:11 httpcore.connection DEBUG    close.started
2024-07-30 15:39:11 httpcore.connection DEBUG    close.complete
2024-07-30 15:39:11 httpcore.connection

100%|██████████| 119/119 [15:44<00:00,  7.94s/it]

2024-07-30 15:39:12 src.functional INFO     filtered 81 samples out of 119 with 4 icl examples
